In [6]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity

def extract_keyframes(video_path, interval=30):
    cap = cv2.VideoCapture(video_path)
    keyframes = []
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if frame_count % interval == 0:  # Trích xuất mỗi 'interval' frame
            keyframes.append(frame)
        success, frame = cap.read()
        frame_count += 1

    cap.release()
    return keyframes

def compare_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

def compare_histogram(frame1, frame2):
    hist1 = cv2.calcHist([frame1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    score = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return score

def compare_features(frame1, frame2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(frame1, None)
    kp2, des2 = orb.detectAndCompute(frame2, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    score = len(matches)  # Số lượng đặc trưng khớp nhau
    return score

def compare_cosine(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    score = cosine_similarity(frame1_gray, frame2_gray)[0][0]
    return score

def compare_videos(video1, video2):
    keyframes1 = extract_keyframes(video1)
    keyframes2 = extract_keyframes(video2)
    
    min_length = min(len(keyframes1), len(keyframes2))

    if min_length == 0:  # Kiểm tra nếu không có khung hình nào để so sánh
        return {
            'SSIM': 0,
            'Histogram': 0,
            'Feature Matching': 0,
            'Cosine Similarity': 0
        }

    total_ssim = 0
    total_hist = 0
    total_features = 0
    total_cosine = 0

    for i in range(min_length):
        frame1 = keyframes1[i]
        frame2 = keyframes2[i]

        total_ssim += compare_ssim(frame1, frame2)
        total_hist += compare_histogram(frame1, frame2)
        total_features += compare_features(frame1, frame2)
        total_cosine += compare_cosine(frame1, frame2)

    average_ssim = total_ssim / min_length if min_length > 0 else 0
    average_hist = total_hist / min_length if min_length > 0 else 0
    average_features = total_features / min_length if min_length > 0 else 0
    average_cosine = total_cosine / min_length if min_length > 0 else 0

    return {
        'SSIM': average_ssim,
        'Histogram': average_hist,
        'Feature Matching': average_features,
        'Cosine Similarity': average_cosine
    }

video1 = 'video_summarization/data/sample_tv360/video_processed/video_processed_4.mp4'
video2 = '/home/thiendc/projects/video_summarization/data/sample_tv360/gt/chunk_4_gt.mp4'

similarity_scores = compare_videos(video1, video2)

print(f"Độ tương đồng trung bình giữa hai video:")
print(f"SSIM: {similarity_scores['SSIM']}")
print(f"Histogram Comparison: {similarity_scores['Histogram']}")
print(f"Feature Matching: {similarity_scores['Feature Matching']}")
print(f"Cosine Similarity: {similarity_scores['Cosine Similarity']}")


Độ tương đồng trung bình giữa hai video:
SSIM: 0
Histogram Comparison: 0
Feature Matching: 0
Cosine Similarity: 0


In [7]:
import cv2
import os

def extract_and_save_keyframes(video_path, output_dir, interval=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    keyframe_count = 0

    # Tạo thư mục lưu trữ khung hình nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)

    success, frame = cap.read()
    
    while success:
        if frame_count % interval == 0:  # Trích xuất mỗi 'interval' khung hình
            keyframe_filename = os.path.join(output_dir, f'keyframe_{keyframe_count:04d}.jpg')
            cv2.imwrite(keyframe_filename, frame)
            keyframe_count += 1
            print(f"Đã lưu: {keyframe_filename}")
        
        success, frame = cap.read()
        frame_count += 1

    cap.release()

# Ví dụ sử dụng
video_path = './data/sample_tv360/gt/chunk_1_gt.mp4'
output_dir = 'gt_keyframe'

extract_and_save_keyframes(video_path, output_dir, interval=30)

Đã lưu: gt_keyframe/keyframe_0000.jpg
Đã lưu: gt_keyframe/keyframe_0001.jpg
Đã lưu: gt_keyframe/keyframe_0002.jpg
Đã lưu: gt_keyframe/keyframe_0003.jpg
Đã lưu: gt_keyframe/keyframe_0004.jpg
Đã lưu: gt_keyframe/keyframe_0005.jpg
Đã lưu: gt_keyframe/keyframe_0006.jpg
Đã lưu: gt_keyframe/keyframe_0007.jpg
Đã lưu: gt_keyframe/keyframe_0008.jpg
Đã lưu: gt_keyframe/keyframe_0009.jpg
Đã lưu: gt_keyframe/keyframe_0010.jpg
Đã lưu: gt_keyframe/keyframe_0011.jpg
Đã lưu: gt_keyframe/keyframe_0012.jpg
Đã lưu: gt_keyframe/keyframe_0013.jpg
Đã lưu: gt_keyframe/keyframe_0014.jpg
Đã lưu: gt_keyframe/keyframe_0015.jpg
Đã lưu: gt_keyframe/keyframe_0016.jpg
Đã lưu: gt_keyframe/keyframe_0017.jpg
Đã lưu: gt_keyframe/keyframe_0018.jpg
Đã lưu: gt_keyframe/keyframe_0019.jpg
Đã lưu: gt_keyframe/keyframe_0020.jpg
Đã lưu: gt_keyframe/keyframe_0021.jpg
Đã lưu: gt_keyframe/keyframe_0022.jpg
Đã lưu: gt_keyframe/keyframe_0023.jpg
Đã lưu: gt_keyframe/keyframe_0024.jpg
Đã lưu: gt_keyframe/keyframe_0025.jpg
Đã lưu: gt_k

In [8]:
video_path = './data/sample_tv360/video_processed/video_processed_1.mp4'
output_dir = 'data_keyframe'
extract_and_save_keyframes(video_path, output_dir, interval=30)

Đã lưu: data_keyframe/keyframe_0000.jpg
Đã lưu: data_keyframe/keyframe_0001.jpg
Đã lưu: data_keyframe/keyframe_0002.jpg
Đã lưu: data_keyframe/keyframe_0003.jpg
Đã lưu: data_keyframe/keyframe_0004.jpg
Đã lưu: data_keyframe/keyframe_0005.jpg
Đã lưu: data_keyframe/keyframe_0006.jpg
Đã lưu: data_keyframe/keyframe_0007.jpg
Đã lưu: data_keyframe/keyframe_0008.jpg
Đã lưu: data_keyframe/keyframe_0009.jpg
Đã lưu: data_keyframe/keyframe_0010.jpg
Đã lưu: data_keyframe/keyframe_0011.jpg
Đã lưu: data_keyframe/keyframe_0012.jpg
Đã lưu: data_keyframe/keyframe_0013.jpg
Đã lưu: data_keyframe/keyframe_0014.jpg
Đã lưu: data_keyframe/keyframe_0015.jpg
Đã lưu: data_keyframe/keyframe_0016.jpg
Đã lưu: data_keyframe/keyframe_0017.jpg
Đã lưu: data_keyframe/keyframe_0018.jpg
Đã lưu: data_keyframe/keyframe_0019.jpg
Đã lưu: data_keyframe/keyframe_0020.jpg
Đã lưu: data_keyframe/keyframe_0021.jpg
Đã lưu: data_keyframe/keyframe_0022.jpg
Đã lưu: data_keyframe/keyframe_0023.jpg
Đã lưu: data_keyframe/keyframe_0024.jpg


In [9]:
import cv2
import os
from skimage.metrics import structural_similarity as ssim

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def compare_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

def count_matching_keyframes(data_folder, gt_folder, threshold=0.9):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    min_length = min(len(data_keyframes), len(gt_keyframes))
    matching_count = 0

    for i in range(min_length):
        frame1 = data_keyframes[i]
        frame2 = gt_keyframes[i]
        
        ssim_score = compare_ssim(frame1, frame2)
        if ssim_score >= threshold:
            matching_count += 1

    return matching_count

# Ví dụ sử dụng
data_folder = 'data_keyframe'
gt_folder = 'gt_keyframe'

matching_keyframes = count_matching_keyframes(data_folder, gt_folder, threshold=0.9)

print(f"Số lượng keyframe giống nhau: {matching_keyframes}")

Số lượng keyframe giống nhau: 15


In [13]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics.pairwise import cosine_similarity
from imagehash import phash
from PIL import Image

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def compare_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

def compare_histogram(frame1, frame2):
    hist1 = cv2.calcHist([frame1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()
    score = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return score

def compare_sift(frame1, frame2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(frame1, None)
    kp2, des2 = sift.detectAndCompute(frame2, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    return len(matches)


def compare_cosine(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY).flatten().reshape(1, -1)
    score = cosine_similarity(frame1_gray, frame2_gray)[0][0]
    return score

def compare_phash(frame1, frame2):
    img1 = Image.fromarray(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
    img2 = Image.fromarray(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
    hash1 = phash(img1)
    hash2 = phash(img2)
    score = 1 - (hash1 - hash2) / len(hash1.hash.flatten())
    return score

def count_matching_keyframes(data_folder, gt_folder, threshold=0.9):
    data_keyframes = load_images_from_folder(data_folder)
    gt_keyframes = load_images_from_folder(gt_folder)

    min_length = min(len(data_keyframes), len(gt_keyframes))
    
    total_ssim = 0
    total_hist = 0
    total_sift = 0
    total_cosine = 0
    total_phash = 0

    for i in range(min_length):
        frame1 = data_keyframes[i]
        frame2 = gt_keyframes[i]

        total_ssim += compare_ssim(frame1, frame2)
        total_hist += compare_histogram(frame1, frame2)
        total_sift += compare_sift(frame1, frame2)
        total_cosine += compare_cosine(frame1, frame2)
        total_phash += compare_phash(frame1, frame2)

    average_ssim = total_ssim / min_length
    average_hist = total_hist / min_length
    average_sift = total_sift / min_length
    average_cosine = total_cosine / min_length
    average_phash = total_phash / min_length

    return {
        'SSIM': average_ssim,
        'Histogram': average_hist,
        'SIFT': average_sift,
        'Cosine Similarity': average_cosine,
        'Perceptual Hash': average_phash
    }

# Ví dụ sử dụng
data_folder = 'data_keyframe'
gt_folder = 'gt_keyframe'

similarity_scores = count_matching_keyframes(data_folder, gt_folder)

print(f"Độ tương đồng giữa hai tập keyframe:")
print(f"SSIM: {similarity_scores['SSIM']}")
print(f"Histogram Comparison: {similarity_scores['Histogram']}")
print(f"SIFT Matching: {similarity_scores['SIFT']}")
print(f"Cosine Similarity: {similarity_scores['Cosine Similarity']}")
print(f"Perceptual Hash: {similarity_scores['Perceptual Hash']}")


Độ tương đồng giữa hai tập keyframe:
SSIM: 0.5614013056559487
Histogram Comparison: 0.6707738222686165
SIFT Matching: 966.8591549295775
Cosine Similarity: 0.7913291550074414
Perceptual Hash: 0.6742957746478874


In [28]:
import numpy as np
from dtaidistance import dtw
from sklearn.metrics.pairwise import cosine_distances

def frames_to_features(frames):
    features = []
    for frame in frames:
        hist = cv2.calcHist([frame], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    return np.array(features)

def euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def cosine_distance(v1, v2):
    v1 = v1.reshape(1, -1)
    v2 = v2.reshape(1, -1)
    return cosine_distances(v1, v2)[0][0]

def compare_frames_with_dtw(data_frames, gt_frames, distance_metric='euclidean'):
    data_features = frames_to_features(data_frames)
    gt_features = frames_to_features(gt_frames)

    distance_matrix = np.zeros((len(data_features), len(gt_features)))

    for i, feature1 in enumerate(data_features):
        for j, feature2 in enumerate(gt_features):
            if distance_metric == 'euclidean':
                distance_matrix[i, j] = euclidean_distance(feature1, feature2)
            elif distance_metric == 'cosine':
                distance_matrix[i, j] = cosine_distance(feature1, feature2)

    dtw_distance = dtw.distance_matrix_fast(distance_matrix)
    return dtw_distance

def average_dtw(dtw_distance):
    total_distance = np.sum(dtw_distance)
    num_elements = dtw_distance.size
    avg_distance = total_distance / num_elements
    return avg_distance

# Ví dụ sử dụng
data_folder = 'data_keyframe'
gt_folder = 'gt_keyframe'

data_frames = load_images_from_folder(data_folder)
gt_frames = load_images_from_folder(gt_folder)

# Tính khoảng cách DTW với Euclidean
dtw_distance_euclidean = compare_frames_with_dtw(data_frames, gt_frames, distance_metric='euclidean')
avg_dtw_euclidean = average_dtw(dtw_distance_euclidean)
print(f"DTW Distance (Euclidean): {dtw_distance_euclidean}")
print(f"Average DTW (Euclidean): {avg_dtw_euclidean}")

# Tính khoảng cách DTW với Cosine
dtw_distance_cosine = compare_frames_with_dtw(data_frames, gt_frames, distance_metric='cosine')
avg_dtw_cosine = average_dtw(dtw_distance_cosine)
print(f"DTW Distance (Cosine): {dtw_distance_cosine}")
print(f"Average DTW (Cosine): {avg_dtw_cosine}")


DTW Distance (Euclidean): [[0.         0.02360181 1.72386861 ... 1.41451762 1.41860983 1.4206369 ]
 [0.02360181 0.         1.72703749 ... 1.40909002 1.41336155 1.41540809]
 [1.72386861 1.72703749 0.         ... 2.75003852 2.75133269 2.75182285]
 ...
 [1.41451762 1.40909002 2.75003852 ... 0.         0.00895269 0.01476587]
 [1.41860983 1.41336155 2.75133269 ... 0.00895269 0.         0.0077151 ]
 [1.4206369  1.41540809 2.75182285 ... 0.01476587 0.0077151  0.        ]]
Average DTW (Euclidean): 1.2295280809475704
DTW Distance (Cosine): [[0.         0.0150933  1.02875303 ... 1.34274343 1.34702966 1.34944011]
 [0.0150933  0.         1.02697443 ... 1.34230004 1.34662647 1.34905536]
 [1.02875303 1.02697443 0.         ... 1.7745021  1.77475408 1.77460677]
 ...
 [1.34274343 1.34230004 1.7745021  ... 0.         0.00932135 0.01699779]
 [1.34702966 1.34662647 1.77475408 ... 0.00932135 0.         0.0090431 ]
 [1.34944011 1.34905536 1.77460677 ... 0.01699779 0.0090431  0.        ]]
Average DTW (Cosine

In [32]:
import cv2
import numpy as np
from dtaidistance import dtw

def load_images_from_folder(folder_path):
    images = []
    image_files = sorted(os.listdir(folder_path))
    for filename in image_files:
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def calculate_mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def frames_to_features(frames):
    features = []
    for frame in frames:
        hist = cv2.calcHist([frame], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    return np.array(features)

def compare_frames_with_dtw(data_frames, gt_frames):
    data_features = frames_to_features(data_frames)
    gt_features = frames_to_features(gt_frames)

    # Chuyển đổi các đặc trưng thành chuỗi đơn giản
    data_seq = np.mean(data_features, axis=1)
    gt_seq = np.mean(gt_features, axis=1)

    # Tính khoảng cách DTW giữa hai chuỗi
    dtw_distance = dtw.distance(data_seq, gt_seq)
    dtw_path = dtw.warping_path(data_seq, gt_seq)

    matched_frames = []
    for i, j in dtw_path:
        if i < len(data_frames) and j < len(gt_frames):
            matched_frames.append((data_frames[i], gt_frames[j]))

    return matched_frames, dtw_distance

def calculate_mse_with_dtw(data_folder, gt_folder):
    data_frames = load_images_from_folder(data_folder)
    gt_frames = load_images_from_folder(gt_folder)

    matched_frames, dtw_distance = compare_frames_with_dtw(data_frames, gt_frames)

    mse_list = []
    for frame1, frame2 in matched_frames:
        if frame1.shape != frame2.shape:
            frame2 = cv2.resize(frame2, (frame1.shape[1], frame1.shape[0]))

        mse = calculate_mse(frame1, frame2)
        mse_list.append(mse)

    avg_mse = np.mean(mse_list)
    return avg_mse, mse_list

# Ví dụ sử dụng
data_folder = 'data_keyframe'
gt_folder = 'gt_keyframe'

avg_mse, mse_list = calculate_mse_with_dtw(data_folder, gt_folder)
print(f"Average MSE giữa các khung hình: {avg_mse}")


Average MSE giữa các khung hình: 16507.09087253687


In [16]:
from difflib import SequenceMatcher

def frames_to_hashes(frames):
    # Chuyển đổi các khung hình thành chuỗi băm để so sánh
    hashes = []
    for frame in frames:
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        hash_value = phash(img)
        hashes.append(hash_value)
    return hashes

def compare_frames_with_lcs(data_frames, gt_frames):
    data_hashes = frames_to_hashes(data_frames)
    gt_hashes = frames_to_hashes(gt_frames)

    # Tính toán LCS giữa hai chuỗi băm
    matcher = SequenceMatcher(None, data_hashes, gt_hashes)
    lcs_length = matcher.find_longest_match(0, len(data_hashes), 0, len(gt_hashes)).size

    return lcs_length

# Ví dụ sử dụng
lcs_length = compare_frames_with_lcs(data_frames, gt_frames)

print(f"Độ dài của chuỗi con chung dài nhất: {lcs_length}")


Độ dài của chuỗi con chung dài nhất: 9
